# 代码提前声明
写代码坚持以下原则：

    *保证高内聚、低耦合的特性，各小块之间均是相互独立，把每个部分集成在一起，随时可以当做不上分点注释掉
    
    *把基本步骤分解开来，数据处理，模型整理都当做可选的方案，整体的可维护性最优
    
    *记住代码的层次、可维护性 是首当其冲，有好的体系下来，就算再难的问题，也能按照大佬模式逐个突破。

# 前置思考方案
    1.特征预处理
       *补缺失：对重要度大的特征进行缺失值补充
       *异常值去除
    ......
    2.单模型尝试
       *lightGBM
       *Catboost
    ......

In [49]:
#################     引包   #####################
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
import matplotlib
import matplotlib.pyplot as plt
import lightgbm as lgb
import operator
import time
import xgboost as xgb

%matplotlib inline

# 一.数据预处理

# ----------------------         1.数据读取        -------------------------

In [50]:
# 读取文件
train_xy = pd.read_csv("./data/train_xy.csv",header=0,sep=",")
train_x = pd.read_csv("./data/train_x.csv",header=0,sep=",")
test_all = pd.read_csv("./data/test_all.csv",header=0,sep=",")
print(train_xy.shape)
print(train_x.shape)
print(test_all.shape)
train = train_xy.copy()
test = test_all.copy()
test['y'] = -1
print('训练集的特征列train：',train.columns)
print('训练集的特征列test：',test.columns)
# 合并一下train 和 test
data = pd.concat([train,test],axis = 0) # train_xy，test_all索引上连接
print(train.shape)
print(test.shape)
print(data.shape)
print('训练集的特征列：',data.columns)

(15000, 160)
(10000, 159)
(10000, 159)
训练集的特征列train： Index(['cust_id', 'cust_group', 'y', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6',
       'x_7',
       ...
       'x_148', 'x_149', 'x_150', 'x_151', 'x_152', 'x_153', 'x_154', 'x_155',
       'x_156', 'x_157'],
      dtype='object', length=160)
训练集的特征列test： Index(['cust_id', 'cust_group', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6',
       'x_7', 'x_8',
       ...
       'x_149', 'x_150', 'x_151', 'x_152', 'x_153', 'x_154', 'x_155', 'x_156',
       'x_157', 'y'],
      dtype='object', length=160)
(15000, 160)
(10000, 160)
(25000, 160)
训练集的特征列： Index(['cust_group', 'cust_id', 'x_1', 'x_10', 'x_100', 'x_101', 'x_102',
       'x_103', 'x_104', 'x_105',
       ...
       'x_91', 'x_92', 'x_93', 'x_94', 'x_95', 'x_96', 'x_97', 'x_98', 'x_99',
       'y'],
      dtype='object', length=160)


d:\python35\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


# ----------------------         2.删除高缺特征        -------------------------
*此处也加入了对缺失统计特征的构建

In [51]:


# 删除一些不必要的特征（噪音、缺失严重、单值、重复等）     删除掉训练集和测试集中同时缺失大于百分之95的
# train ,test 分开分析 
# 处理一下缺失值严重的特征列，删除
def get_nan_feature(train,rate = 0.95):
    total_num = train.shape[0]
    train_nan_feats = []
    for i in range(157):
        feat = 'x_' + str(i+1)
        nan_num = train.loc[train[feat]==-99,:].shape[0]
        nan_rate = nan_num / float(total_num) 
        
        if nan_rate == 1.0:                    # 只有nan
            train_nan_feats.append(feat)
        if nan_rate > rate:                    # 有缺失值 nan,而且缺失严重
            if len(train[feat].unique()) == 2: # 只有nan + 一个其他值
                train_nan_feats.append(feat)

    print("一共有 %d 个特征列的缺失值严重，超过%f "%(len(train_nan_feats),rate))
    return train_nan_feats

train_nan_feats = get_nan_feature(train)
test_nan_feats = get_nan_feature(test)
print("缺失严重的特征：train =?= test------",np.all(train_nan_feats == test_nan_feats))

# 对这些特征取并集:28个
nan_feats = list(set(train_nan_feats) | set(test_nan_feats)) # 按照train | test的结果,并集，交集，A or B 都一样     并集方式的，其实可以多尝试
print('严重缺失的特征有 %d 个。'%(len(nan_feats)))

# 总的删除的特征（发现删重复的5个特征，效果不好，所以只删除28个缺失严重的特征,这是有尝试过得）
drop_feats = nan_feats 
print('一共删除的特征有 %d 个。'%(len(drop_feats)))
print(drop_feats)

# 删除缺失值严重的特征列
train = train.drop(drop_feats, axis = 1)
test = test.drop(drop_feats, axis = 1)
data = data.drop(drop_feats, axis = 1)
print(data.shape)
print(train.shape)
print(test.shape)
# 删除的特征，全部是特征重要性=0的特征，所以删除 = 原始的特征（不影响精确效果，但是可以减少噪音）
# 删除了x_92 , x_94 是数值型的，其他 24 个 全部是 类别型

print('训练集的特征列：',data.columns)

一共有 19 个特征列的缺失值严重，超过0.950000 
一共有 26 个特征列的缺失值严重，超过0.950000 
缺失严重的特征：train =?= test------ False
严重缺失的特征有 28 个。
一共删除的特征有 28 个。
['x_116', 'x_127', 'x_126', 'x_129', 'x_133', 'x_131', 'x_135', 'x_112', 'x_109', 'x_113', 'x_110', 'x_94', 'x_102', 'x_92', 'x_134', 'x_137', 'x_119', 'x_118', 'x_128', 'x_115', 'x_132', 'x_107', 'x_111', 'x_130', 'x_138', 'x_123', 'x_108', 'x_114']
(25000, 132)
(15000, 132)
(10000, 132)
训练集的特征列： Index(['cust_group', 'cust_id', 'x_1', 'x_10', 'x_100', 'x_101', 'x_103',
       'x_104', 'x_105', 'x_106',
       ...
       'x_9', 'x_90', 'x_91', 'x_93', 'x_95', 'x_96', 'x_97', 'x_98', 'x_99',
       'y'],
      dtype='object', length=132)


# ----------------------         3.对缺失特征做处理        -------------------------

   *此处还附加了将数值型特征进行排序并归一化处理，相当于做成了排序特征
   
   *也附加了构建缺失统计特征，并对缺失统计进行onehot编码，让其更稳定

   *挑选重要度大、缺失比较少的特征进行填充

In [52]:

# 对剩下的特征进行分析，分为数值型 、 类别型        (这里有疑问，难道默认0-94列特征是数值型特征吗，这很不合理啊)
numerical_features = []
categorical_features = []
for i in range(157):
    feat = "x_" + str(i+1)
    if feat not in drop_feats:
        if i <= 94: # 1-95
            numerical_features.append(feat)
        else:
            categorical_features.append(feat)
print("剩下的数值型特征：",len(numerical_features))
print("剩下的类别型特征：",len(categorical_features))

print('数值型特征为：',categorical_features)
print('数值型为：',data[categorical_features])
print('训练集的特征列：',data.columns)


剩下的数值型特征： 93
剩下的类别型特征： 36
数值型特征为： ['x_96', 'x_97', 'x_98', 'x_99', 'x_100', 'x_101', 'x_103', 'x_104', 'x_105', 'x_106', 'x_117', 'x_120', 'x_121', 'x_122', 'x_124', 'x_125', 'x_136', 'x_139', 'x_140', 'x_141', 'x_142', 'x_143', 'x_144', 'x_145', 'x_146', 'x_147', 'x_148', 'x_149', 'x_150', 'x_151', 'x_152', 'x_153', 'x_154', 'x_155', 'x_156', 'x_157']
数值型为：       x_96  x_97  x_98  x_99  x_100  x_101  x_103  x_104  x_105  x_106  ...    \
0        2     2     2     1      1      1    -99    -99    -99    -99  ...     
1        1     2     1     1      1      1    -99    -99    -99    -99  ...     
2        1     1     1     1      1      1    -99    -99    -99    -99  ...     
3        1     2     2     1      1      1    -99    -99    -99    -99  ...     
4        2     1     1     1      1      1    -99    -99    -99    -99  ...     
5        2     2     2     1      1      1    -99    -99    -99    -99  ...     
6        1     1     1     1      1      1    -99    -99    -99    -99  

In [53]:
# 统计每个样本缺失值的个数                  统计缺失样本数 构建统计特征
def get_nan_count(data,feats,bins = 7):
    df = data[feats].copy()
    df = df.replace(-99,np.nan)
    print('总列数:',df.shape[1])   #这列展示了每一行缺失的特征的数量
    print('每行非空列数:',df.count(axis = 1).values)   #这列展示了每一行缺失的特征的数量
    df['nan_count'] = df.shape[1] - df.count(axis = 1).values  # 列数 - 非nan数
    print('每行空列数:',df['nan_count'])   #这列展示了每一行缺失的特征的数量
    dummy = pd.get_dummies(pd.cut(df['nan_count'],bins),prefix = 'nan') #把每行空列数，做7分离散化再转one-hot编码 对缺失数据进行离散化,划分为7个区间,对于划分区间，这里根据空值情况来造dummies特征
    print(dummy.shape)
    res = pd.concat([data,dummy],axis = 1) # 合并到原来的数据
    print(res.shape)
    return res
# 在全部特征上面统计缺失值      新加入了对缺失值统计的7列
data = get_nan_count(data,data.columns.values,7)
print('训练集的特征列：',data.columns)

总列数: 132
每行非空列数: [ 84  85 120 ... 117  82  85]
每行空列数: 0       48
1       47
2       12
3       11
4       29
5       54
6       15
7       51
8       11
9       48
10      32
11      11
12      30
13      50
14      15
15      33
16      15
17      33
18      51
19      51
20      12
21      48
22      12
23      13
24      15
25      52
26      12
27      12
28      29
29      15
        ..
9970    15
9971    15
9972    14
9973    51
9974    51
9975    12
9976    11
9977    12
9978    48
9979    16
9980    15
9981    11
9982    15
9983    12
9984    51
9985    51
9986    15
9987    13
9988    14
9989    50
9990    14
9991    51
9992    12
9993    48
9994    15
9995    14
9996    11
9997    15
9998    50
9999    47
Name: nan_count, Length: 25000, dtype: int64
(25000, 7)
(25000, 139)
训练集的特征列： Index(['cust_group', 'cust_id', 'x_1', 'x_10', 'x_100', 'x_101', 'x_103',
       'x_104', 'x_105', 'x_106',
       ...
       'x_98', 'x_99', 'y', 'nan_(7.895, 23.0]', 'nan_(23.0, 38.0]',
       'n

In [54]:
# 获取缺失很少的数值型的特征         缺失少的数值型用均值
def get_little_nan_feats(df,numerical_features,rate = 0.1):
    total_num = df.shape[0]
    little_nan_feats = []
    for feat in numerical_features:
        nan_num = df.loc[df[feat]==-99,:].shape[0]
        nan_rate = nan_num / float(total_num) 
        if nan_rate <= rate:
            little_nan_feats.append(feat)
            #print("feature:",feat,"nan_num = ",nan_num,"nan_rate = ",nan_rate)
    print("一共有 %d 个特征列的缺失值较少，低于%f "%(len(little_nan_feats),rate))
    return little_nan_feats
little_nan_feats = get_little_nan_feats(data, numerical_features)
print("缺失较少的特征：",little_nan_feats)

print('训练集的特征列：',data.columns)
'''
# 对一些数值的特征进行平均值填充
for feat in little_nan_feats:
    data[feat] = data[feat].replace(-99,np.nan)
    data[feat] = data[feat].fillna(data[feat].mean())
'''# 对一些重要数值的特征进行平均值填充    (根据其特性吧，因为不是时序问题，线性差值肯定是不合适的。)
#对重要特征进行填充
for feat in ['x_81','x_95']:
    data[feat] = data[feat].replace(-99,np.nan)
    data[feat] = data[feat].fillna(data[feat].mean())


# 对数值型的特征，处理为rank特征（鲁棒性好一点）     数值本身就代表大小的意思，这里构建排序特征并进行归一化，效果会更加鲁棒一些。

for feat in numerical_features:
    #print('rank前：',data[feat])
    data[feat] = data[feat].rank() / float(data.shape[0]) # 排序，并且进行归一化        这样也行？
    #print('rank后：',data[feat])
print('训练集的特征列：',data.columns)

一共有 55 个特征列的缺失值较少，低于0.100000 
缺失较少的特征： ['x_1', 'x_2', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58', 'x_59', 'x_60', 'x_61', 'x_62', 'x_63', 'x_64', 'x_65', 'x_66', 'x_67', 'x_68', 'x_69', 'x_70', 'x_71', 'x_72', 'x_73', 'x_74', 'x_75', 'x_76', 'x_77', 'x_78', 'x_79', 'x_80', 'x_81', 'x_82', 'x_83', 'x_84', 'x_85', 'x_86', 'x_87', 'x_88', 'x_89', 'x_90', 'x_91']
训练集的特征列： Index(['cust_group', 'cust_id', 'x_1', 'x_10', 'x_100', 'x_101', 'x_103',
       'x_104', 'x_105', 'x_106',
       ...
       'x_98', 'x_99', 'y', 'nan_(7.895, 23.0]', 'nan_(23.0, 38.0]',
       'nan_(38.0, 53.0]', 'nan_(53.0, 68.0]', 'nan_(68.0, 83.0]',
       'nan_(83.0, 98.0]', 'nan_(98.0, 113.0]'],
      dtype='object', length=139)
训练集的特征列： Index(['cust_group', 'cust_id', 'x_1', 'x_10', 'x_100', 'x_101', 'x_103',
       'x_104', 'x_105', 'x_106',
       ...
       'x_98', 'x_99', 'y', 'nan_(7.895, 23.0]'

In [55]:
#对重要的特征进行填充，之后还会想着对着重要的特征利用其进行采样操作
imp_feat=['x_80','x_2','x_81','x_95','x_1','x_52','x_63','x_54','x_43','x_40','x_93','x_42','x_157','x_62','x_29','x_61','x_55']
print('假定的重要特征个数为：',len(imp_feat))
for feat in imp_feat[:10]:
    if feat in numerical_features:
        print('进行填充吧')
        data[feat]=data[feat].replace(-99,np.nan)
        data[feat]=data[feat].fillna(data[feat].mean())
    if feat in categorical_features:
        print('这是类别特征：',feat)
        
        
#确认下这些特征是否已经被填充好了
for feat in ['x_80','x_2','x_81','x_95','x_1','x_52']:
    nan_num = data.loc[data[feat]==-99,:].shape[0]
    print('缺失数量为：',nan_num)

假定的重要特征个数为： 17
进行填充吧
进行填充吧
进行填充吧
进行填充吧
进行填充吧
进行填充吧
进行填充吧
进行填充吧
进行填充吧
进行填充吧
缺失数量为： 0
缺失数量为： 0
缺失数量为： 0
缺失数量为： 0
缺失数量为： 0
缺失数量为： 0


# ----------------------         4.数据采样处理        -------------------------

In [56]:
#这个方式可能效果不太好

# from imblearn.over_sampling import SMOTE       #过度抽样处理库SMOTE
# from imblearn.under_sampling import RandomUnderSampler
# #进行两种采样方式的处理，欠采样和过采样的处理，尝试在重要的10个特征上面进行采样，来获得标签
# #过采样处理\
# print('开始了,第一种方式(可选择：选择计算的距离依据是最重要的哪几个特征？ 往往更多的特征依据会起到更好的采样生成效果，可以多做尝试吧 )')
# x= data[['x_80','x_2','x_81','x_95','x_1','x_52','x_63','x_54','x_43']]
# feature_columns=[i for i in x.columns]
# y =data['y']
# #groupby_data_orginal = y.groupby('label').count()
# #print(x)
# #print(y)
# model_smote = SMOTE()  # 建立smote模型对象
# x_smote_resampled, y_smote_resampled = model_smote.fit_sample(x, y)
# x_smote_resampled = pd.DataFrame(x_smote_resampled, columns=feature_columns)
# y_smote_resampled = pd.DataFrame(y_smote_resampled, columns=['y'])
# smote_resampled = pd.concat([x_smote_resampled, y_smote_resampled], axis=1)
# groupby_data_smote = smote_resampled.groupby('y').count()
# smote_resampled.to_csv('./feature_data/经过过采样后的数据.csv')
# print(smote_resampled)

# 二.特征工程

# ----------------------         1.特征整理      -------------------------

In [57]:
######################################################      1.特征整理   #############################################

train = data.loc[data['y']!=-1,:] # train set
test = data.loc[data['y']==-1,:]  # test set
print(train.shape)
print(test.shape)
#print(train)
print('训练集的特征列：',train.columns)
# 获取特征列，去除id，group,y
no_features = ['cust_id','cust_group','y'] 
features = [feat for feat in train.columns.values if feat not in no_features]
print("所有特征的维度：",len(features))

(15000, 139)
(10000, 139)
训练集的特征列： Index(['cust_group', 'cust_id', 'x_1', 'x_10', 'x_100', 'x_101', 'x_103',
       'x_104', 'x_105', 'x_106',
       ...
       'x_98', 'x_99', 'y', 'nan_(7.895, 23.0]', 'nan_(23.0, 38.0]',
       'nan_(38.0, 53.0]', 'nan_(53.0, 68.0]', 'nan_(68.0, 83.0]',
       'nan_(83.0, 98.0]', 'nan_(98.0, 113.0]'],
      dtype='object', length=139)
所有特征的维度： 136


# ----------------------         2.构建交叉特征       -------------------------

In [58]:
######################################################        #############################################

#使用找到的还不错的特征，但是发现线上效果并不好，所以就不用了吧

#多项式特征，二次变换特征    变换特征基本没什么用
# train['all_index']=[i for i in range(0,15000)]
# train['x_80-x_63']=train['x_80']-train['x_63']
# train['x_80+x_64']=train['x_80']+train['x_64']
# train['x_80/x_63']=train['x_80']/train['x_63']

# train['x_95+x_93']=train['x_95']+train['x_42']
# train['x_80+x_93']=train['x_80']+train['x_42']
# train['x_63*x_93']=train['x_95']-train['x_42']
# train['x_63**2']=train['x_80']**1/2


# #test['all_index']=[i for i in range(0,10000)]

# test['x_95+x_93']=test['x_95']+test['x_42']
# test['x_80+x_93']=test['x_80']+test['x_42']
# test['x_63*x_93']=test['x_95']-test['x_42']
# test['x_63**2']=test['x_80']**1/2

# test['x_80-x_63']=test['x_80']-test['x_63']
# test['x_80+x_64']=test['x_80']+test['x_64']
# test['x_80/x_63']=test['x_80']/test['x_63']

# #这里，之前交叉特征根本没加入到训练里面尴尬
# features.extend(['x_95+x_93','x_80+x_93','x_63*x_93','x_63**2','x_80-x_63','x_80+x_64','x_80/x_63'])
# print(train.columns)

# ----------------------         3.特征分离       -------------------------

In [59]:

# 得到输入X ，输出y
#train=train.replace(-99,np.nan)#  尝试进行替换之后再训练看看效果（发现效果变差了）
#test=test.replace(-99,np.nan)#  尝试进行替换之后再训练看看效果（发现效果变差了）
train_id = train.pop('cust_id')
y = train['y'].values
X = train[features].values
print('X features :',features)
print("X shape:",X.shape)
print("y shape:",y.shape)

test_id = test.pop('cust_id')
test_data = test[features].values
print("test shape",test_data.shape)
print(train.columns)
train.to_csv('./feature_data/经过特征工程后的整个表.csv')
# X.to_csv('./feature_data/train_feature.csv')
# y.to_csv('./feature_data/train_label.csv')
print('训练集的特征列：',train.columns)

X features : ['x_1', 'x_10', 'x_100', 'x_101', 'x_103', 'x_104', 'x_105', 'x_106', 'x_11', 'x_117', 'x_12', 'x_120', 'x_121', 'x_122', 'x_124', 'x_125', 'x_13', 'x_136', 'x_139', 'x_14', 'x_140', 'x_141', 'x_142', 'x_143', 'x_144', 'x_145', 'x_146', 'x_147', 'x_148', 'x_149', 'x_15', 'x_150', 'x_151', 'x_152', 'x_153', 'x_154', 'x_155', 'x_156', 'x_157', 'x_16', 'x_17', 'x_18', 'x_19', 'x_2', 'x_20', 'x_21', 'x_22', 'x_23', 'x_24', 'x_25', 'x_26', 'x_27', 'x_28', 'x_29', 'x_3', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 'x_37', 'x_38', 'x_39', 'x_4', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 'x_49', 'x_5', 'x_50', 'x_51', 'x_52', 'x_53', 'x_54', 'x_55', 'x_56', 'x_57', 'x_58', 'x_59', 'x_6', 'x_60', 'x_61', 'x_62', 'x_63', 'x_64', 'x_65', 'x_66', 'x_67', 'x_68', 'x_69', 'x_7', 'x_70', 'x_71', 'x_72', 'x_73', 'x_74', 'x_75', 'x_76', 'x_77', 'x_78', 'x_79', 'x_8', 'x_80', 'x_81', 'x_82', 'x_83', 'x_84', 'x_85', 'x_86', 'x_87', 'x_88', 'x_89', 'x

# 三.单模型方式

# ----------------------         1.Xgboost模型       -------------------------

In [60]:
#开始xgb的cv训练
# 采取分层采样
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
print("start：********************************")
start = time.time()

N = 5
skf = StratifiedKFold(n_splits=N,shuffle=True,random_state=2018)

auc_cv = []
pred_cv = []
for k,(train_in,test_in) in enumerate(skf.split(X,y)):
    X_train,X_test,y_train,y_test = X[train_in],X[test_in],\
                                    y[train_in],y[test_in]    
    # 模型参数
    params = {	'booster':'gbtree',
	'objective': 'binary:logistic',
	'early_stopping_rounds':100,
#     'gamma':0,#0.2 is ok
#     'max_depth':8,
# # 	'lambda':550,
    'subsample':0.7,
    'colsample_bytree':0.3,
    'min_child_weight':2.5, 
    'eta': 0.007,
	'seed':2018,
	'nthread':7}
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvali = xgb.DMatrix(X_test)
    model = xgb.train(params, dtrain, num_boost_round=800)
    predict = model.predict(dvali)
    tmp_auc = roc_auc_score(y_test,predict)
    auc_cv.append(tmp_auc)
    print("valid auc:",tmp_auc)
    # test
    pred = model.predict(xgb.DMatrix(test_data))
    pred_cv.append(pred) 
    
    
#     xgb.plot_importance(model,max_num_features=20)
#     plt.title('xgb Feature Importance')
#     plt.xlabel('relative importance')
#     plt.show()

# K交叉验证的平均分数 
print('the cv information:')
print(auc_cv)
print('cv mean score',np.mean(auc_cv))

end = time.time()
print("......................run with time: ",(end - start) / 60.0 )
print("over:*********************************")

start：********************************
valid auc: 0.8148273833763178
valid auc: 0.7920578494819676
valid auc: 0.8613744315822521
valid auc: 0.836001985031244
valid auc: 0.7894928802638178
the cv information:
[0.8148273833763178, 0.7920578494819676, 0.8613744315822521, 0.836001985031244, 0.7894928802638178]
cv mean score 0.8187509059471199
......................run with time:  0.7499267260233561
over:*********************************


--------------------------------------------------------------------------------------------------------------------------------------
线下效果记录：（记住那些特征能够在线上涨分就作为使用的特征）


    [1]使用如下参数，cv效果是0.8095941001927958   (初版特征)
              'booster': 'gbtree',    
              'objective': 'binary:logistic',
              'eta': 0.02,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,  # 0.8
              'subsample': 0.7,
              'min_child_weight': 9,  # 2 3
              'silent': 1
    [2]默认参数下，cv值是：0.6694547520305475
    
    
    [3]使用如下调参后的结果为0.8118425236460529   (初版特征)
              'booster': 'gbtree',
              'objective': 'binary:logistic',
              'eta': 0.02,
              'max_depth': 3,  # 4 3         (调出)
              'colsample_bytree': 0.7,  # 0.8
              'subsample': 0.5,              (调出)
              'min_child_weight': 9,  # 2 3
              'silent': 1
    [4]使用如下调参后的结果为0.814163899785226    (初版特征)
              'booster': 'gbtree',
              'objective': 'binary:logistic',
              'eta': 0.01,
              'max_depth': 3,  # 4 3
              'colsample_bytree': 0.5,  # 0.8
              'subsample': 0.5,
              'min_child_weight': 9,  # 2 3
              'silent': 1,
              'gamma': 0    
     [5]使用模型4与lightgbm模型(0.81952101)进行blend （前期放弃融合方案，做单模型计算，并且每个模型都要充分考虑过拟合的问题）
     0.7    0.3的权值分配  线下：0.8184459400734287     线上值：0.75182 （这个稳定性稍好点）
     0.6    0.4的权值分配  线下：0.818504121445845
     0.5    0.5的权值分配  线下：0.8182861386523573    
     0.9    0.1的权值分配  线下：0.8175251371900479
     与lighhtgbm模型（0.8211793803301299）结合线下：  
     0.7    0.3的权值分配  线下：0.8210557752290836     线上值：0.750+效果变差了
     [6]使用如下调参后的结果为0.8187509059471199    (初版特征)
     'booster':'gbtree',
	'objective': 'binary:logistic',
	'early_stopping_rounds':100,
    'subsample':0.7,
    'colsample_bytree':0.3,
    'min_child_weight':2.5, 
    'eta': 0.007,
	'seed':2018,
	'nthread':7
     注意：与高cv的线下分进行融合效果变差，说明调参调的过拟合了。
     ！！！！！！！！！！！！！！！！！！！！！！！！！！！！   不做模型调整了，   ！！！！！！！！！！！！！！！！！！！！
     [7]把-99替换下为nan,让模型自己去判断空值 在线下有点提升为0.8190473478328284
     [8]加入交叉特征，线下为0.8192969307017934  ，统计特征之间加入没效果，是自己加入方式不对，有问题，没有把列名实际加上去，程序逻辑写的出问题了。 线上有下降，那么交叉特征就干脆不用了
       train['x_80-x_63']=train['x_80']-train['x_63']
       train['x_80+x_64']=train['x_80']+train['x_64']
       train['x_80/x_63']=train['x_80']/train['x_63']
       train['x_95+x_93']=train['x_95']+train['x_42']
       train['x_80+x_93']=train['x_80']+train['x_42']
       train['x_63*x_93']=train['x_95']-train['x_42']
       train['x_63**2']=train['x_80']**1/2
   
     [9]使用上序号，能够让线下效果提升到0.9842626010230955，但是这样根本就不行哎，会在线上完全过拟合的，(用统计特征会降分，又过拟合了)
        train['all_index']=[i for i in range(0,15000)]
        train['x_80-x_63']=train['x_80']-train['x_63']
        train['x_80+x_64']=train['x_80']+train['x_64']
        train['x_80/x_63']=train['x_80']/train['x_63']

        train['x_95+x_93']=train['x_95']+train['x_42']
        train['x_80+x_93']=train['x_80']+train['x_42']
        train['x_63*x_93']=train['x_95']-train['x_42']
        train['x_63**2']=train['x_80']**1/2
     [10]原效果是0.8190，现在对最重要的10个特征进行填充，线下效果变成0.8190473478328284,发现并没有多大的效果，但是这种提分点可能是在xgboost中不敏感吧。
        
        
参数包括：
参考文献：https://blog.csdn.net/han_xiaoyang/article/details/52665396
--------------------------------------------------------------------------------------------------------------------------------------

In [61]:
#生成预测结果
mean_auc = np.mean(auc_cv)
print("mean auc:",mean_auc)
filepath = 'result/xgb_'+ str(mean_auc)+'.csv' # 线下平均分数

# 转为array
res =  np.array(pred_cv)
print("总的结果：",res.shape)

# 最后结果，mean，max，min
r = res.mean(axis = 0)
print('result shape:',r.shape)

result = DataFrame()
result['cust_id'] = test_id
result['pred_prob'] = r
result.to_csv(filepath,index=False,sep=",")

mean auc: 0.8187509059471199
总的结果： (5, 10000)
result shape: (10000,)


In [ ]:
#xgb找参数（网格调参）
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
#使用网格搜索找最佳的参数
param_grid = {'gamma':[0,0.1,0.2,0.3,],'learning_rate':[0.01,0.02,0.03,0.04,],'colsample_bytree':[0.5,0.6,0.7,0.8],'subsample':[0.5,0.6,0.7,0.8]}
gsearch1 = GridSearchCV(estimator = XGBClassifier(max_depth=3,
    learning_rate =0.01, n_estimators=140,       
    min_child_weight=1,objective= 'binary:logistic', nthread=4,seed=27),       
    param_grid=param_grid,cv=5)
gsearch1.fit(X,y)
print(gsearch1.best_params_,gsearch1.best_score_)

--------------------------------------------------------------------------------------------------------------------------------

相比于xgboost中的方式，会有些参数名字上的修改，不过影响不大：
    
    1、eta -> learning_rate 
    2、lambda -> reg_lambda 
    3、alpha -> reg_alpha

调参记录:

    *boostergbtree 树模型做为基分类器（默认）
    *gbliner 线性模型做为基分类器
    *n_jobs  并行线程数
    *silentsilent=0时，不输出中间过程（默认）silent=1时，输出中间过程
    *nthreadnthread=-1时，使用全部CPU进行并行运算（默认）nthread=1时，使用1个CPU进行运算。
    *scale_pos_weight正样本的权重，在二分类任务中，当正负样本比例失衡时，设置正样本的权重，模型效果更好。例如，当正负样本比例为1:10时scale_pos_weight=10。
    *n_estimatores含义：总共迭代的次数，即决策树的个数调参：
    *max_depth含义：树的深度，默认值为6，典型值3-10。调参：值越大，越容易过拟合；值越小，越容易欠拟合。
    *min_child_weight含义：默认值为1,。调参：值越大，越容易欠拟合；值越小，越容易过拟合（值较大时，避免模型学习到局部的特殊样本）。
    *subsample含义：训练每棵树时，使用的数据占全部训练集的比例。默认值为1，典型值为0.5-1。调参：防止overfitting。
    *colsample_bytree含义：训练每棵树时，使用的特征占全部特征的比例。默认值为1，典型值为0.5-1。调参：防止overfitting。
    *learning_rate含义：学习率，控制每次迭代更新权重时的步长，默认0.3。调参：值越小，训练越慢。典型值为0.01-0.2。
    *gamma惩罚项系数，指定节点分裂所需的最小损失函数下降值。调参：
    *alphaL1正则化系数，默认为1lambdaL2正则化系数，默认为1


# 四.模型融合

# 附录1：尝试点日志

# 附录2：上分点日志